<h1>Use of Bayes theorom and n-grams to classify (score) passwords.</h1>

Bayes theory provides us with the formula:

$$P(A|B)=\frac{P(A)\prod_{i=1}^n P(B|A)_i}{\prod_{i=1}^n P(B)_i}$$

Since we are interested in scoring passwords based on content of n-grams, we can assign meaning:

<ul>
<li>$P(A)$ = probability that a word is valid</li>
<li>$P(B)$ = probability that a word - valid or not! - contains a (set of) digrams</li>
<li>$P(A|B)$ = probability that a word is valid given it contains a particular (set of) digrams.</li>
<li>$P(B|A)$ = probability that a particular (set of) digrams is contained in a valid word</li>
</ul>

And this is probably where my entire theory unravels... by substitution, we can arrive at:

$$P(wordvalid|setofngrams)=\frac{P(wordvalid)\prod_{i=1}^n P(containsngram_i|wordvalid)_i}{\prod_{i=1}^n P(containsngram_i)_i}$$

<h2>The Alphabet</h2>
To begin our analysis, we begin with an alphabet 'A' (a list) consisting of 'a' (an integer) characters:

<ul>
<li>a = size of alphabet used to build words (int)</li>
<li>A = alphabet (list)</li>
</ul>

For an alphabet with an 'a' of 3, A would be ['a', 'b', 'c'].

<h2>The Words</h2>
Building from the alphabet, we have two word lists:

The first list is all known valid words.  We will refer to this as 'D' (a list), which contains 'd' distinct words.  This is the training data.

The second list is all possible words which can be built from the alphabet given other constraints.  For a word of length $l_w$ (represented by the variable 'lw'), we can arrange the letters of the alphabet into $a^lw$ possible permutations.  We will call this list 'W', and it contains 'w' (an integer) entries.

In most cases, 'W' will be difficult or impossible to compute fully.  By comparison, the dictionary (D) is a small subset of this list of possible words (W).

<ul>
<li>D = list of known valid words (the dictionary)</li>
<li>d = the number of words in 'D'</li>
<li>W = list of all possible words (the world)</li>
<li>w = the number of words in 'W'</li>
<li>$l_w$ or 'lw' is the length of a word built from 'A'
</ul>

<h2>N-Grams</h2>

N-Grams are groupsings of 'n' adjacent letters taken from a word.  For instance, the set of 2-grams (more commonly called 'bigrams') from the word 'alpha' would be ['al', 'lp', 'ph', 'ha'].

<ul>
<li>$l_n$ (an integer) is the size of n-gram.  In the code, this is variable 'ln'.</li>
<li>G (a list) is the set of all NGrams which can be built using alphabet 'A'
<li>g (an integer) is the number of distinct NGrams in the list G.
</ul>

In processing the dictionary (D), ngram frequencies are collected for each word in a frequency table 'F':

<ul>
<li>F (a dict) is the frequency of ngrams from all words in the dictionary<li>
</ul>

Finally, for a word of length $l_w$, we can see that $g_w = l_w - l_n + 1$ ngrams can be extracted.

<h2>The Math</h2>
Returning to our equiation at the top of this frame,

<ul>
<li>$P(A) = d/w$.  Probability that a word is valid, this will be 'P_wv' in the code below.</li>
<li>$P(B)$ = probability of a word (valid or not) containing a particular set of ngrams.</li>
<li>$P(A|B)$ = the &#36;1,000,000 question!</li>
<li>$P(B|A)$ = the product of all ngram probabilities, each taken from F.</li>
</ul>

Given that 'W' is impossible (or, more accurately, infeasible) to compute, building a frequency table for it as we have done with 'D' and 'F' will be equally difficult.  Still, these values are required to compute P(B).

I also believe that this is the point at which my work begins to falter.

Since W is all permutations of A, we have a possible work-around.  We know how many permutations should exist if we actually built W.  We will call this quantity 'w' (an integer).  We also know how many ngrams will be produced for a word of a given length (gw) and the number of ngrams which can be produced by the alphabet A (eg, g). Together,

$$c = \frac{g_w * w}{g}$$

The use of the name 'c' here is arbitrary, but it represents a count of the number of times a random ngram appears across all of W.

$$P(B) = (\frac{c}{w})^{l_w}$$


In [3]:
## Set global variables

# this is the length of word we're targeting
lw = 6

# this is the size of digram we're using.  2 is too short.  3 may be appropriate.
ln = 3

# this is the file which contains "valid" words.  Use a reasonable default.  See
# https://wiki.skullsecurity.org/Passwords for a list of real-world passwords.
dictionary = '/usr/share/dict/words'

# defines the alphabet (valid characters).  Flip these booleans to True/False
# and the alphabet will be constructed properly below.
use_upper = True
use_lower = True
use_numer = False
use_punct = False

# enable verbose output in calculations (for debugging)
verbose=True

In [4]:
# Okay... we'll do all the setup in this cell.  Should be no need for
# modifications here.

from math import pow
import re

# create the alphabet
A=''
if use_upper is True: A += 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
if use_lower is True: A += 'abcdefghijklmnopqrstuvwxyz'
if use_numer is True: A += '0123456789'
if use_punct is True: A += '!@#$%^&*.,?:\-'  # note -- incomplete set!!  Be careful
                                             # about breaking the regex below.

a = int(len(A))       # size of the alphabet
w = int(pow(a, lw))   # number of words in the universe (a^lw)
g = int(pow(a, ln))   # number of possible ngrams built from A
gw = ((lw - ln) + 1)  # number of ngrams in a word

print('Number of ngrams per word: {}'.format(gw))
print('Words in universe (a = {}, lw = {}) = {}'.format(a, lw, w))
print('Possible ngrams (a = {}, ln = {}) = {}'.format(a, ln, g))

## Build the dictionary

# regex used to filter words (see output below after execution!)
rtmp = '^[{}]{{{}}}$'.format(A, lw)
regex = re.compile(rtmp)
print('Using filter regex: ' + rtmp)

D = []
dictfile = open(dictionary, 'r')
for line in dictfile:
    word = line.strip()
    if regex.match(word): D.append(word)

d = len(D)
print('{} words read'.format(d))

Number of ngrams per word: 4
Words in universe (a = 52, lw = 6) = 19770609664
Possible ngrams (a = 52, ln = 3) = 140608
Using filter regex: ^[ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz]{6}$
9146 words read


In [5]:
## extract frequencies

from ngram import NGram

ng = NGram(N=ln)
ngrams = dict()  # the dictionary containing count of all grams

for word in D:
    grams = list(ng.ngrams(word))  # only count unique ngrams
    for gram in grams:
        try:
            ngrams[gram] = ngrams[gram] + 1
        except KeyError:
            ngrams[gram] = 1

print('Unique ngrams extracted: {}'.format(len(ngrams) ))
print('Possible ngrams not observed: {}'.format(g - len(ngrams)))

Unique ngrams extracted: 5167
Possible ngrams not observed: 135441


In [7]:
def wordValid(inword):
    if verbose is True: print('input word is: {0:s}'.format(inword))

    P_wv = float(d) / float(w)  # probability of a word being valid
    if verbose is True: print('P(wv) = {0:d} / {1:d} = {2:.20f}'.format(d, w, P_wv))

    P_ng_wv = 1  # probability of a ngram sequence given word is valid (eg, in dictionary)
    if verbose is True: print('Ngrams with frequencies for {0:s}'.format(inword))
    for gram in list(ng.ngrams(inword)):
        try:
            occs = ngrams[gram]
        except KeyError:
            occs = 0
            
        if verbose is True: print('  P({0:s}) = {1:d} / {2:d} = {3:.20f}'.format(gram, occs, d, float(occs) / float(d)))
        P_ng_wv *= (float(occs) / float(d))

    if verbose is True: print('P(ng|wv) = {0:.20f}'.format(P_ng_wv))
    
    c = (gw * w) / g
    P_ng = pow((float(c) / float(w)), gw)
    if verbose is True: print('P(ng) = ({0:d} / {1:d})^{2:d} = {3:.20f}'.format(c, w, gw, P_ng))

    P = (P_wv * P_ng_wv) / P_ng        
    return P

# a handful of sample test sets for various word lengths (may need adjustment!!)
testsets = { 3: ['cac', 'bac', 'cab' ],
             6: ['shaggy', 'purple', 'zaxbys', 'Latino', 'queasy' ],
             8: ['AJNAUTH', 'onuraage', '1n6sd11y', 'MINNIVER', 'PFELDMAN', 'ATLANTA9', 
                 'erenberg', 'rnjoyhuh', 'GIS-MOLL', 'jenalynn', 'alalalal', 'F*&vsQ2r'] }

testset = testsets[lw]

for word in testset:
    print('{0:s}: {1:20f}'.format(word, wordValid(word)))
    print('')

input word is: shaggy
P(wv) = 9146 / 19770609664 = 0.00000046260586574899
Ngrams with frequencies for shaggy
  P(sha) = 43 / 9146 = 0.00470150885633063668
  P(hag) = 2 / 9146 = 0.00021867483052700635
  P(agg) = 21 / 9146 = 0.00229608572053356663
  P(ggy) = 5 / 9146 = 0.00054668707631751580
P(ng|wv) = 0.00000000000129051472
P(ng) = (562432 / 19770609664)^4 = 0.00000000000000000065
shaggy:             0.911537

input word is: purple
P(wv) = 9146 / 19770609664 = 0.00000046260586574899
Ngrams with frequencies for purple
  P(pur) = 25 / 9146 = 0.00273343538158757945
  P(urp) = 6 / 9146 = 0.00065602449158101901
  P(rpl) = 2 / 9146 = 0.00021867483052700635
  P(ple) = 35 / 9146 = 0.00382680953422261105
P(ng|wv) = 0.00000000000150059851
P(ng) = (562432 / 19770609664)^4 = 0.00000000000000000065
purple:             1.059927

input word is: zaxbys
P(wv) = 9146 / 19770609664 = 0.00000046260586574899
Ngrams with frequencies for zaxbys
  P(zax) = 0 / 9146 = 0.00000000000000000000
  P(axb) = 0 / 9146 